In [25]:
%run ./"0 Imports and functions.ipynb"

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
spectral_cube 0.6.1.dev22+g003ef16
spectral_cube file path /home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
reproject 0.8
1.23.1 Numpy
astrodendro_file: /home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
Results will be saved to Directory ./Result Files


In [5]:
# Pointing_Information

Pointing_Information = {}

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "HCN_J1-0.cube.fits" #the name of the initial SC file.
File_Descriptor="NGC_253_HCN_J1-0_"
Pointing_Information["File_Descriptor"] = "NGC_253_HCN_J1-0_"
Pointing_Information["target"] = "NGC253"#or the cmz
Pointing_Information["center"] = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs') #the center of NGC253

desired_beam_size = 4.3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 3.5*u.Mpc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=33*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=78*u.deg
Pointing_Information["target_velocity"]=250*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 3.4*u.km/u.s
ovs = 3 #how much do you desire to oversample the beam by
desired_FOV = [70,360]*u.pc# pc normal to the disk, across the disk

print("done")

gc.collect

done


<function gc.collect(generation=2)>

In [6]:

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header

try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Informat    del scCopy
ion['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 
    
beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec)


Pointing_Information["original_BMAJ"]=beam_major
Pointing_Information["original_BMIN"]=beam_minor
Pointing_Information["original_BMAJ_pc"]=beam_major.to(u.rad)*Pointing_Information['distance']
Pointing_Information["original_BMIN_pc"]=beam_minor.to(u.rad)*Pointing_Information['distance']
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size


#this accounts for elliptical beams:            
Pointing_Information['original_pixel_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_pixel_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_spatial_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']
Pointing_Information['original_spatial_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']#convert to pc using the distance

average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

Pointing_Information['original_beam_oversampling_MAJ'] = beam_major/average_pixel
Pointing_Information['original_beam_oversampling_MIN'] = beam_minor/average_pixel
Pointing_Information['desired_beam_oversampling'] = ovs
Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])


Center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> Pixel center: 577 648 Pixel FOV: [66, 342]
cropping cube. rotation: 33.0 deg center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> crop to: [ 70. 360.] pc
cropped cube from [<Quantity 0. pc>, <Quantity 0. pc>] to: [ 70. 360.] pc
Center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> Pixel center: 577 648 Pixel FOV: [66, 342]
cropping cube. rotation: 33.0 deg center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> crop to: [ 70. 360.] pc
cropped cube from [<Quantity 0. pc>, <Quantity 0. pc>] to: [ 70. 360.] pc


# 1. Reprojection

In [26]:
gc.collect()

#
# def Reproject_To_Region(Pointing_Information,Cube_Information,i_step=30,Cube_Name_Save='',Force_Origin=[False,[0,0]*u.deg,[0,0]*u.deg]):

Reproject_To_Region(Pointing_Information,Cube_Information,i_step=30,Cube_Name_Save='',Force_Origin=[False,[0,0]*u.deg,[0,0]*u.deg])
    
# This is the cube name that they are being saved to unless you changed it inside the reproject functions definiton

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

gc.collect()

print("done")

begin step: 0 of 7
start beam convolution
0.0 1.4368850473684927
2.2800652280150217e-09 1.0864614693810945e-10
convolve end

fov crop start 

Center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> Pixel center: 577 648 Pixel FOV: [66, 342]
cropping cube. rotation: 33.0 deg center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> crop to: [ 70. 360.] pc
(22, 1221, 1221) 407 889 255 899
cropped cube from [<Quantity 0. pc>, <Quantity 0. pc>] to: [ 70. 360.] pc
fov cropped

start reprojection

check max SC value: 0.002141568413172502 SC shape: (22, 482, 644)

end reprojection

check max SC value: 0.002103109952898464 SC shape: (22, 177, 261)
fov crop start 2 

Center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> Pixel center: 117 87 Pixel FOV: [24, 125]
cropping cube. rotation: 33.0 deg center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> crop to: [ 70. 360.] pc
(22, 177, 261) 1 174 1 234
cropped cube from [<Quantity 921

# 1a. Splice the reprojected cubes back together

In [27]:
# First, splice the partwise cubes back together

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Splice_vels(Cube_Name_Load=Cube_Name_Save)

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

gc.collect()

print("done")

Loaded 0_4.3pc_beam_NGC_253_HCN_J1-0_70.0x360.0pc_reprojected.fits
Loaded 1_4.3pc_beam_NGC_253_HCN_J1-0_70.0x360.0pc_reprojected.fits
Loaded 2_4.3pc_beam_NGC_253_HCN_J1-0_70.0x360.0pc_reprojected.fits
Loaded 3_4.3pc_beam_NGC_253_HCN_J1-0_70.0x360.0pc_reprojected.fits
Loaded 4_4.3pc_beam_NGC_253_HCN_J1-0_70.0x360.0pc_reprojected.fits
Loaded 5_4.3pc_beam_NGC_253_HCN_J1-0_70.0x360.0pc_reprojected.fits
[Errno 2] No such file or directory: 'Spectral Cubes/6_4.3pc_beam_NGC_253_HCN_J1-0_70.0x360.0pc_reprojected.fits'
Center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> Pixel center: 116 86 Pixel FOV: [24, 125]
cropping cube. rotation: 33.0 deg center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> crop to: [ 70. 360.] pc
(153, 173, 233) 1 172 1 232
cropped cube from [<Quantity 900. pc>, <Quantity 584.5 pc>] to: [ 70. 360.] pc
done


# 1b. Velocity reprojection

In [28]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-6]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

Center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> Pixel center: 116 86 Pixel FOV: [24, 125]
cropping cube. rotation: 33.0 deg center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> crop to: [ 70. 360.] pc
(153, 173, 233) 1 172 1 232
cropped cube from [<Quantity 900. pc>, <Quantity 584.5 pc>] to: [ 70. 360.] pc
Start velocity reprojection of 4.3pc_beam_NGC_253_HCN_J1-0_70.0x360.0pc_reprojected.fits
Smoothed to Gaussian Kernel of width 0.3415279232372052
Wrote reprojected cube to Spectral Cubes/4.3pc_beam_NGC_253_HCN_J1-0_70.0x360.0pc_reprojecte_3.4_vel_res_.fits


0

# 1c. Fix repeated pixels

In [29]:
# after the reprojection, there may be duplicated data from the fact that it just fills in stuff 
# expecting new data to be there. This fixes it by checking to see if it matches other data, then removes
# the repeated slices. in other words, it may be set to reproject to 0-500 km/s through the velocity channels
# but there may only be data between 100-400, so it will fill it in with copied data channels. This will check for those.



# Update to the current cube name

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-6]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Run the function to crop the nan values out of the cube

Remove_Repeated_Pixels(Cube_Name_Save)

Center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> Pixel center: 116 86 Pixel FOV: [24, 125]
cropping cube. rotation: 33.0 deg center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> crop to: [ 70. 360.] pc
(148, 173, 233) 1 172 1 232
cropped cube from [<Quantity 900. pc>, <Quantity 584.5 pc>] to: [ 70. 360.] pc
Good data starting from channel 0 ; start has been cropped to that channel
Good data ending at channel 147 ; end has been cropped to that channel
Cropped cube saved as  Spectral Cubes/Cropped_4.3pc_beam_NGC_253_HCN_J1-0_70.0x360.0pc_reprojecte_3.4_vel_res_.fits


# x. save cube information

In [36]:
# Now that all the data reduction is concluded, save information about the new cubes

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = "Cropped_"+Cube_Name_Save[0:len(Cube_Name_Save)-6]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

File_Descriptor="NGC_253_HCN_J1-0_"

import pickle 

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Cube_Information", 'wb') as file:
    pickle.dump(Cube_Information, file)

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Pointing_Information", 'wb') as file:
    pickle.dump(Pointing_Information, file)


Center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> Pixel center: 116 86 Pixel FOV: [24, 125]
cropping cube. rotation: 33.0 deg center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> crop to: [ 70. 360.] pc
(147, 173, 233) 1 172 1 232
cropped cube from [<Quantity 900. pc>, <Quantity 584.5 pc>] to: [ 70. 360.] pc
